First Dataset from: https://www.kaggle.com/datasets/rmisra/clothing-fit-dataset-for-size-recommendation
Second Dataset from: https://www.kaggle.com/datasets/tourist55/clothessizeprediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
import textblob

In [106]:
df_modcloth = pd.read_json('./data/modcloth_final_data.json', lines = True)
df_modcloth.shape

(82790, 18)

df_modcloth = df_modcloth[(df_modcloth["category"] == "tops") | 
    (df_modcloth["category"] == "bottoms") | 
    (df_modcloth["category"] == "dresses") | 
    (df_modcloth["category"] == "outerwear")]

In [107]:
df_modcloth['category'].value_counts()

new          21488
tops         20364
dresses      18650
bottoms      15266
outerwear     4223
sale          2524
wedding        275
Name: category, dtype: int64

In [108]:
df_modcloth["fit"].value_counts()

fit      56757
large    13059
small    12974
Name: fit, dtype: int64

In [88]:
dict_fit = {
    "fit"   : 0,
    "small" : -1,
    "large" : 1,
}

In [89]:
df_modcloth["fit"] = df_modcloth["fit"].map(lambda x: dict_fit[x])


In [90]:
df_modcloth.columns

Index(['item_id', 'waist', 'size', 'quality', 'cup size', 'hips', 'bra size',
       'category', 'bust', 'height', 'user_name', 'length', 'fit', 'user_id',
       'shoe size', 'shoe width', 'review_summary', 'review_text'],
      dtype='object')

In [97]:
series_text = df_modcloth["review_text"].reset_index(drop = True)

In [98]:
series_fit = df_modcloth["fit"].reset_index(drop = True)

In [100]:
# Natural Language Processing libraries, initiations and functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
import re # Delete this if scraping in same notebook

def preprocess(text):
    text = re.sub(r'[^a-zA-Z]',' ', text.lower())
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 1 and not word in stop_words])
cvec = CountVectorizer(analyzer = "word",
                       min_df = 4,
                       preprocessor = preprocess,
                       stop_words = 'english')
#pd.DataFrame(cvec.fit_transform(df['COLUMN']).todense(), columns=cvec.get_feature_names())

def Sentamentize(text):
    return TextBlob(str(text)).sentiment.polarity

In [101]:

cvec_text = cvec.fit_transform(series_text.fillna(' '))
df_cvec = pd.DataFrame(cvec_text.todense(),columns=cvec.vocabulary_)

C:\Users\Peridot of Earth\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [102]:
df_cvec.head(20)

,dress,amazing,arrived,today,love,absolutely,sunny,girl,line,different,...,upstate,steve,madden,feltlike,parka,offcenter,pea,polar,vortex,brokerage
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [103]:
import spacy

ModuleNotFoundError: No module named 'spacy'

## Copied Code
Code copied from: https://www.youtube.com/watch?v=CzRrD76pnVY

In [70]:
df_mod_only = df_modcloth.drop(["item_id", "size", "quality", "cup size", 
                                "hips", "bra size", "category", "height", "user_id",
                                "fit", "review_summary"], axis = 1)

In [71]:
df_mod_only.dropna(inplace = True)

In [73]:
series_length = df_mod_only["length"]

In [82]:
df_mod_words = pd.concat([series_length, text_df])

In [83]:
df_mod_words

,0,aa,ab,abc,abd,abdomen,abdominal,abide,ability,abit,...,zillion,zip,zipped,zipper,zippered,zipping,zipup,zone,zoo,zoom
8490,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8491,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8492,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8493,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8494,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58496,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58497,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58498,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58499,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Clean Code
# Reset Index
# 

In [3]:
df_rent = pd.read_json('./data/renttherunway_final_data.json', lines = True)

In [4]:
df_rent.shape

(192544, 15)

In [7]:
df_rent.iloc[:100000].to_csv("./data/renttherunway_first.csv")

In [8]:
df_rent.iloc[100000:].to_csv("./data/renttherunway_last.csv")

In [10]:
pd.read_csv("./data/renttherunway_last.csv")

,Unnamed: 0,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,100000,large,896669,36b,222318,140lbs,8.0,other,Got this for a baby shower but it ran a bit la...,athletic,Nice Fabric but Size down,shift,"5' 7""",8,42.0,"June 1, 2016"
1,100001,small,759150,34b,2776206,155lbs,8.0,everyday,"I loved this blazer, but I should have sized u...",pear,Unique!,blazer,"5' 5""",12,39.0,"February 24, 2016"
2,100002,large,462822,34b,1570915,135lbs,8.0,work,Love the classic look and quality material. Bu...,hourglass,"Pretty dress, great for the office, but WAY ov...",dress,"5' 4""",12,32.0,"March 17, 2016"
3,100003,fit,384182,36b,1406056,145lbs,10.0,other,Rented this dress to help out with a friend's ...,athletic,Gorgeous sexy dress!,dress,"5' 3""",14,43.0,"July 1, 2017"
4,100004,fit,424201,34d,1313942,145lbs,10.0,date,I wore this dress for my birthday dinner and i...,full bust,Great dress for birthday dinner in the winter!,dress,"5' 5""",12,28.0,"March 21, 2016"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92539,192539,fit,66386,34dd,2252812,140lbs,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,"5' 9""",8,42.0,"May 18, 2016"
92540,192540,fit,118398,32c,682043,100lbs,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,"5' 1""",4,29.0,"September 30, 2016"
92541,192541,fit,47002,36a,683251,135lbs,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,"5' 8""",8,31.0,"March 4, 2016"
92542,192542,fit,961120,36c,126335,165lbs,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,"5' 6""",16,31.0,"November 25, 2015"


In [ ]:

df.dtypes# Return column data types
df.describe() #(returns count, mean, std, min, and %iles)???
df.info()
df['col1'].std() # Standard Deviation


# Main ways to adjust indexes
df.reset_index(inplace=True)
df.set_index('column', inplace = True)
df = df.reindex(['cols'], axis = 1)



# Concat
the_dfs = [df_one,df_two]
df_main = pd.concat(the_dfs, sort=False, axis = 1)

# Dropping columns with "Errors"
mask = df['col'] == 'Error'
df.drop(df[mask].index, inplace = True)

# Dropping Duplicates
df.drop_duplicates(inplace = True)

# Map Function Starter
def map_funct(cell):
    return cell
df['column'] = df['column'].map(lambda x: map_funct(x))


# Masks
mask = df['column'] <=> something 
df = df[mask]

df[~mask] (Select opposite of the mask)
df[(df['col1'] > num) & (df['col2'] < num)]
df[(df['col1'] > num) | (df['col2'] > num)]
df[ df['col'].isin([num, num]) ] #return rows with specific values(?)

### EDA

In [ ]:
# Basic EDA

df.dtypes                           # Data Types of cols
df.column.unique()                  # Unique Values in col
df.describe().T                     # Summary Stats
df.info                             # Info
df['column'].value_counts(normalize=True)

In [ ]:
| Thing             | Main Matplotlib               | Sub Plt               | Seaborn                           |
|-------------------|-------------------------------|-----------------------|-----------------------------------|
| Idea              | + to plt each line            |fig, ax = plt.subplots(ncols, nrows) (ax[#,#] for ea plt)| |
| Bar               | plt.bar(x, height)            |                       |                                   |
| Histagram         | plt.hist(x, bins)(Lists Args) |                       |                                   |
| Scatter           | plt.scatter(x, y)    (Args)   |                       |                                   |
| Line Graph        | plt.plot([y's_ea_x(#,#)(#,])  | ax[#]plot()           | sns.lineplot(x,y)(Args)           |
| FacetGrid (Chi2)  |                               |                       | sns.FacetGrid(df, col, row)       |
| ...define inr plts|                               |                       |.map(plt.graph, bins, range)       |
| X/Y Labels        | plt.zlabel("")                | ax[#].set_xlabel()    |                                   |
| Title             | plt.title("")                 | ax[#].set_title()     |                                   |
| Figure Size       | plt.figure(figsize)           | plt.subplots(figsize) |                                   |
|Customize X/Y Ticks| plt.zticks([labels], rotation)|
| Add a Line        | plt.axvline(x)                |
| Set Z Value Limit | plt.gca().set_zlim([#,#])     |
| Add Legend        | plt.legend()                  |


The six "lessons" of visualizing data.
 - Lesson 1: Understand the context. (Who, What, How)
 - Lesson 2: Choose an appropriate visual display. (What type of graph?)
 - Lesson 3: Eliminate clutter. (What can I delete or make more subtle?)
 - Lesson 4: Focus attention where you want it. (How do I emphasize important things?)
 - Lesson 5: Think like a designer. (How do I organize my visualization?)
 - Lesson 6: Tell a story! (How do I communicate my visualization?)


-------------------------------------------

## Set Up Options

# import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Color Options
colors = {'blue': '#729ECE',
          'brown': '#A8786E',
          'green': '#67BF5C',
          'grey': '#A2A2A2',
          'orange': '#FF9E4A',
          'pink': '#ED97CA',
          'purple': '#AD8BC9',
          'red': '#ED665D',
          'teal': '#6DCCDA',
          'yellow': '#CDCC5D'}

# Style Options
sns.set_style('darkgrid')
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')


# Basic Non-Visual EDA

df.dtypes                           # Data Types of cols
df.column.unique()                  # Unique Values in col
df.describe().T                     # Summary Stats
df.info                             # Info
df['column'].value_counts(normalize=True)

# Correlations between two seperate columns w/o a whole thing
print(np.corrcoef(df['column_1'].values, df['column_2'].values))

# SCATTER


## PLT
plt.figure(figsize=(10, 10))
plt.scatter(y = df['column_one'],
            x = df['column_two'],
            color = colors);

## PD
df.plot('math', 'num_awards', kind = 'scatter')


## SNS Dirty
axes = plt.gca()
axes.set_xlim([-1,35])
axes.set_ylim([-1,35])
sns.scatterplot(df['column_one'], df['column_two'], hue= df['column_three'], alpha = 0.1);

# BOXPLOT

## SNS
sns.boxplot(df['column'],
            data = df
                orient='v' or 'h',
                fliersize=8,
                linewidth=#,
                notch = True,
                saturation=0.5,
                ax=ax,
                by = "column?");

# LINEPLOT

sns.lineplot(data = df);

====================================================
# HISTAGRAM

## PLT
plt.hist(df['column'], bins = 10, color='g', alpha = 0.1);
plt.xlabel("Label", position = (0,0), ha = 'left', color = 'grey') # (x,y)
plt.ylabel("label", position = (0, 1), ha = 'right', color = 'grey'); # (x,y);

## PLT.plot
figure, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (15,15))
test_df['column_one'].plot(ax = ax[0], kind = 'hist')
test_df['column_two'].plot(ax = ax[1], kind = 'hist');

## PD
df.plot(x='col1', y='col2', title="Title")
df.hist('col');

## SNS Countplot
sns.countplot(raw_df['col']);
##Also
sns.countplot(x="col", data=df);

------------
# Other Example DIRTY
plt.hist(df['col1'],bins= len(df['col1'].unique()));


====================================================
# BARPLOT

## PLT
plt.figure(figsize = (16,6))
plt.bar(height = df['Column?'],  # List of numbers?
        x = df.columns);

## SNS
sns.barplot(data = df,
            x = "column_one",
            y = "column_two");




====================================================
# PAIRPLOT

sns.pairplot(df, kind = "reg", 
             hue = 'column',
             plot_kws = {'line_kws' : {'color' : 'red'}, 
                         'scatter_kws' : {'alpha' : 0.1}})
             # Alt
             # plot_kws={'alpha': 0.05});

# With specific rows, only
sns.pairplot(data=temp_df,
            x_vars=['col1'],
            y_vars=['col2'])


====================================================
# HEATMAP

# SNS
#plt.figure(figsize= (9,9))
sns.heatmap(df.corr(), 
            cbar=False, 
            linewidths= .01, 
            linecolor='black', 
            cmap=sns.color_palette("Purples"),
            annot=True);


=======================================================
# Jointplot
sns.jointplot(y_test, lin_reg.predict(X_test))

=======================================================
# Facet Grid
Grid = sns.FacetGrid(df, col="Col1", row="col2", margin_titles=True)
Grid.map(plt.hist,'col2?', bins = num, range = (num,num));

=======================================================
# Graph Alterations
# Colors: plt.colors.cnames

# For layering on top:
plt.hist(df['column_1'], color='g')
plt.hist(df['column_2'], color='b')

# For layering next to:
fig, ax = plt.subplots(2, 
                    sharex = True,  # Scale x together
                    sharey = True,    
                   figsize = (1,1)) # Size
ax[0].hist(df['column_1'],          # 
           bins = 10,               # Bins
           color='b',               # Color
           alpha = 0.5,             # Transparancy
           label = 'col_1')         # For .legend()
ax[1].hist(df['column_2'])          # 

# Subplot Example

fig, axes = plt.subplots(3, figsize = (20,10))
sns.countplot(df['col'], ax= axes[0])
sns.countplot(df['col'], ax = axes[1])
sns.countplot(df['col'], ax = axes[2]);

# Labels for Words
plt.xlabel("measure",               # Alt: .ylabel
           position = (0,0),        # (x-pos, y-pos)?
           ha = 'left',             # Horizontal
           color = 'grey') # (x,y)

plt.legend()
plt.title()                         # Title ~= Subtitle
plt.suptitle("subtitle\n.",         # Subtitle
             position = (0,1),      # Position
             ha = 'left',           # Horizontal
             fontsize=16,           # Size, Font
             va = 'top');           # Vertical


# Shifting Graph Coordinates (in a Scatter plot)
axes = plt.gca()
axes.set_xlim([num_one,num_two])
axes.set_ylim([num_one,num_two])
sns.scatterplot(df['column_one'], df['column_two']);

In [51]:
df_modcloth = df_modcloth.drop(["user_name", "shoe size", "shoe width"], axis = 1)

In [52]:
df_modcloth = df_modcloth.drop(["waist", "bust"], axis = 1)

In [53]:
def fun_height_to_inches(str_height):
    if type(str_height) != str:
        inches_out = str_height
    else:
        str_height = str_height.replace("in","")
        str_feet = str_height.split("ft")[0]
        str_inch = str_height.split("ft")[1]
        if str_inch == '':
            inches_out = (int(str_feet) * 12)
        else:
            inches_out = (int(str_feet) * 12) + int(str_inch)
    return inches_out

df_modcloth["height"] = df_modcloth["height"].map(lambda x: fun_height_to_inches(x))

In [54]:
for col in df_modcloth.columns:
    print(col, (df_modcloth[col].isna().sum()/df_modcloth.shape[0]).round(5))

item_id 0.0
size 0.0
quality 0.0008
cup size 0.07685
hips 0.31997
bra size 0.07447
category 0.0
height 0.01311
length 0.00041
fit 0.0
user_id 0.0
review_summary 3e-05
review_text 3e-05


In [55]:
df_modcloth.shape

(58503, 13)

In [56]:
dict_length ={
    "very short"    : -2,
    "slightly short": -1,
    "just right"    :  0,
    "slightly long" :  1,
    "very long"     :  2
}
df_modcloth["length"] = df_modcloth["length"].fillna("just right").map(lambda x: dict_length[x])
df_modcloth["length"].value_counts()

just right        43285
slightly long      8425
slightly short     4418
very long          1767
very short          584
Name: length, dtype: int64